In [1]:
# this model is based on attention is all you need paper

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

In [3]:
torch.set_default_device('cuda') # sets the default device for everytensor and model to GPU

In [4]:
class MultiHeadAttention(nn.Module):
    
    # the model is defined by the dimensions of the model and the number of heads that it has
    
    # the method that initializes the MultiHeadAttention Layer
    
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        
        # the dimension of weights under each head
        
        self.d_k = d_model // num_heads
        
        # the weight matrices to convert the inputs to queries, keys and values. 
        # also the W_o matrix for generating the output.
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        # if mask is given we then label the corresponding attention score to minus infinity
        
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        
        # the dim = -1 is used to specify the dimension along which the softmax is to be applied.
        # as the softmax normalizes the values after taking them to the exponential, it requires 
        # the dimension along which the normalization takes place (basically the dim that decides
        # the sum of the denominator of the normalization) 
        
        attn_probs = torch.softmax(attn_scores, dim=-1)
       
        # the output is weighted values (weighted using attention)
    
        output = torch.matmul(attn_probs, V)
        return output
    
    def split_heads(self, x):
        batch_size, sequence_length, d_model = x.size()
        
        # x.view is essentially the same as reshape in numpy.
        
        return x.view(batch_size, sequence_length, self.num_heads, self.d_k).transpose(1, 2)
    
    def combine_heads(self, x):
        batch_size, _, sequence_length, d_k = x.size()
        
        return x.transpose(1, 2).contiguous().view(batch_size, sequence_length, self.d_model)
    
    # this is the default function for running a layer/model inherited from nn.Module
    
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        
        return output

In [5]:
# this is a position independent layer and its name has no meaning for any position related computation

class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        
        self.relu = nn.ReLU()
        
        
    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [6]:
# this is responsible for understanding the position of the words/tokens in the data sequence

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_sequence_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_sequence_length, d_model)
        position = torch.arange(0, max_sequence_length, dtype=torch.float).unsqueeze(1)
        denmn_term = torch.exp(torch.arange(0, d_model, 2).float() * -math.log(10000.0) / d_model)
        
        pe[:, 0::2] = torch.sin(position * denmn_term)
        pe[:, 1::2] = torch.cos(position * denmn_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [7]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attention = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attention_output = self.self_attention(x, x, x, mask)
        x = self.norm1(x + self.dropout(attention_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [8]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attention = MultiHeadAttention(d_model, num_heads)
        self.cross_attention = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attention_output = self.self_attention(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attention_output))
        
        # the query comes from the same decoder and the key and value comes from the encoder
        
        attention_output = self.cross_attention(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attention_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        
        return x

In [9]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)
        
        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        
        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)
    
    
    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        
        sequence_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, sequence_length, sequence_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask
    
    
    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(tgt)))
        
        enc_output = src_embedded
        for encoder_layer in self.encoder_layers:
            enc_output = encoder_layer(enc_output, src_mask)
            
        dec_output = tgt_embedded
        for decoder_layer in self.decoder_layers:
            dec_output = decoder_layer(dec_output, enc_output, src_mask, tgt_mask)
            
        output = self.fc(dec_output)
        return output

In [10]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

In [11]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=1e-3, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(100):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:,:-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:,1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 8.694465637207031
Epoch: 2, Loss: 8.5112943649292
Epoch: 3, Loss: 8.205148696899414
Epoch: 4, Loss: 8.70296859741211
Epoch: 5, Loss: 8.208598136901855
Epoch: 6, Loss: 8.164833068847656
Epoch: 7, Loss: 8.130943298339844
Epoch: 8, Loss: 8.107088088989258
Epoch: 9, Loss: 8.06690788269043
Epoch: 10, Loss: 7.896693229675293
Epoch: 11, Loss: 7.911427974700928
Epoch: 12, Loss: 7.955441474914551
Epoch: 13, Loss: 7.628218650817871
Epoch: 14, Loss: 7.652629375457764
Epoch: 15, Loss: 7.5073771476745605
Epoch: 16, Loss: 7.4711737632751465
Epoch: 17, Loss: 7.185066223144531
Epoch: 18, Loss: 6.960652828216553
Epoch: 19, Loss: 6.768046855926514
Epoch: 20, Loss: 6.5784783363342285
Epoch: 21, Loss: 6.463686943054199
Epoch: 22, Loss: 6.140863418579102
Epoch: 23, Loss: 5.914595127105713
Epoch: 24, Loss: 5.524152755737305
Epoch: 25, Loss: 5.437939167022705
Epoch: 26, Loss: 5.099974632263184
Epoch: 27, Loss: 5.068725109100342
Epoch: 28, Loss: 4.886218547821045
Epoch: 29, Loss: 4.58107566833